In [1]:
# import sys
# !{sys.executable} -m spacy download en_core_web_sm

In [2]:
import glob
import pandas as pd
from tabula import read_pdf
from tika import parser 
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.tokens import Doc
import gensim
from gensim.models.ldamodel import LdaModel
import gensim.corpora as corpora
from gensim import models
import spacy
import re

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
nlp.max_length = 1500000

In [3]:
files1 = glob.glob("/Users/test/Desktop/topic modelsss/new/Session 72 - 2017/*.PDF")
files2 = glob.glob("/Users/test/Desktop/topic modelsss/new/Session 72 - 2017/*.pdf")

In [4]:
docs = []

for file in files1:
    raw = parser.from_file(file)
    doc = raw['content'].split()
    docs.append(doc)

for file in files2:
    raw = parser.from_file(file)
    doc = raw['content'].split()
    docs.append(doc)

In [5]:
def alphanumeric(word):
    return re.sub(r'\W+', '', word)

In [6]:
def contains_non_ascii(s):
    return any(ord(i) > 127 for i in s)

def is_valid_word(word, stop_words=STOP_WORDS):
    """Checks if the word is valid for training or inference"""

    if stop_words and word in stop_words:
        return False

    if contains_non_ascii(word):
        return False

    distinct_keys = len(''.join(set(word)))
    if len(word) <= 2 or distinct_keys == 1:
        return False

    if word.isdigit() or word == "0":
        return False

    return True


In [7]:
def spacy_lemma(vocab_str):

    for i in nlp(vocab_str):
        return i.lemma_

In [8]:
docs_concated = []
for doc in docs:
    docs_concated = docs_concated + doc

In [9]:
df = pd.DataFrame({'vocab':docs_concated})

In [10]:
df['vocab'] = df['vocab'].str.lower()

In [11]:
df['vocab'] = df['vocab'].apply(alphanumeric)

In [12]:
df['valid'] = df['vocab'].apply(is_valid_word)
df = df[df['valid']==True]
df = df.reset_index(drop=True)

In [15]:
# df['vocab'] = df['vocab'].apply(spacy_lemma)

In [13]:
vocab = df.groupby("vocab").size().reset_index(name='count').sort_values("count", ascending=False).head(4000)

In [14]:
vocab = set(vocab['vocab'].values)

In [15]:
def keep_top_words(doc, vocab=vocab):
    return [word for word in doc if word in vocab]

In [16]:
docs_cleaned = [keep_top_words(doc) for doc in docs]

In [46]:
id2word = corpora.Dictionary(docs_cleaned)
corpus = [id2word.doc2bow(text) for text in docs_cleaned]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]


In [61]:
lda_model = gensim.models.LdaModel(corpus_tfidf, id2word=id2word,  iterations=150, alpha=0.0001, num_topics=10)

In [62]:
for idx, topic in lda_model.print_topics(-1):
        print('Topic: {} \nWords: {}'.format(idx, topic))


Topic: 0 
Words: 0.001*"lift" + 0.001*"corrupt" + 0.001*"devotion" + 0.001*"wait" + 0.001*"ignorance" + 0.001*"seats" + 0.001*"games" + 0.001*"die" + 0.001*"represented" + 0.001*"enemies"
Topic: 1 
Words: 0.001*"budgets" + 0.001*"murdered" + 0.001*"violated" + 0.001*"apartheid" + 0.001*"drugs" + 0.001*"patience" + 0.001*"drug" + 0.001*"hydrogen" + 0.001*"correspondent" + 0.001*"occupation"
Topic: 2 
Words: 0.001*"indigenous" + 0.001*"aspect" + 0.001*"formal" + 0.001*"lourish" + 0.001*"escorted" + 0.001*"rescue" + 0.001*"grand" + 0.001*"yesterday" + 0.001*"crop" + 0.001*"nutrition"
Topic: 3 
Words: 0.001*"environments" + 0.001*"fisheries" + 0.001*"repatriation" + 0.001*"escorted" + 0.001*"disregard" + 0.001*"centuries" + 0.001*"serves" + 0.001*"minorities" + 0.001*"committee" + 0.001*"reforming"
Topic: 4 
Words: 0.001*"bank" + 0.001*"tax" + 0.001*"escorted" + 0.001*"provinces" + 0.001*"apartheid" + 0.001*"definition" + 0.001*"terrorists" + 0.001*"victim" + 0.001*"electoral" + 0.001*"era